<a href="https://colab.research.google.com/github/Nashi8/Prompt-Engineering/blob/main/PE_Customer_Service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install --quiet openai

from google.colab import userdata
from openai import OpenAI
import json
from typing import Dict

api_key = userdata.get("118A_OpenAI_API")

if api_key is None:
    raise ValueError("Secret '118A_OpenAI_API' not found. Check Colab → Settings → Secrets.")

client = OpenAI(api_key=api_key)

print(" OpenAI client initialized.")

 OpenAI client initialized.


In [16]:
def call_llm(
    system_prompt: str,
    user_prompt: str,
    model: str = "gpt-4.1-mini",
    temperature: float = 0.3,
) -> str:
    """
    Thin wrapper around the OpenAI chat completion API.
    """
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=temperature,
    )
    return response.choices[0].message.content.strip()

In [17]:
def classify_issue(user_message: str) -> str:
    """
    Step 1: Classify the issue into a simple category.
    """
    system_prompt = "You are an AI that categorizes customer support messages."
    user_prompt = f"""
    Read the customer's message and respond with exactly one label:
    - billing
    - technical
    - account
    - other

    Respond with ONLY the label.

    Customer message:
    \"\"\"{user_message}\"\"\"
    """
    category = call_llm(system_prompt, user_prompt)
    return category.strip().lower()


def extract_details(user_message: str, category: str) -> Dict[str, str]:
    """
    Step 2: Extract structured details as JSON.
    """
    system_prompt = "You are an AI that extracts structured details from customer issues."
    user_prompt = f"""
    You are given a customer message and its category: {category}.

    Extract key details as JSON with these keys:
    - "summary": short 1–2 sentence summary of the issue
    - "important_details": list of important details (as a bulleted markdown string)
    - "urgency": one of ["low", "medium", "high"]

    Respond with ONLY valid JSON (no backticks, no explanation).

    Customer message:
    \"\"\"{user_message}\"\"\"
    """
    details_json = call_llm(system_prompt, user_prompt)
    details = json.loads(details_json)
    return details


def generate_support_reply(user_message: str, category: str, details: Dict[str, str]) -> str:
    """
    Step 3: Generate a friendly final reply using the structured info.
    """
    system_prompt = "You are a friendly, concise customer support agent."
    user_prompt = f"""
    Write a helpful reply to the customer.

    Constraints:
    - Be empathetic and concise
    - Use plain language
    - Suggest clear next steps and, if relevant, what your team will do

    Context:
    - Category: {category}
    - Extracted summary: {details.get("summary")}
    - Important details (markdown list): {details.get("important_details")}
    - Urgency: {details.get("urgency")}

    Customer message:
    \"\"\"{user_message}\"\"\"
    """
    reply = call_llm(system_prompt, user_prompt)
    return reply


def customer_support_flow(user_message: str) -> Dict[str, object]:
    """
    Runs the full prompt chain and returns everything
    so you can inspect intermediate steps.
    """
    category = classify_issue(user_message)
    details = extract_details(user_message, category)
    reply = generate_support_reply(user_message, category, details)

    return {
        "category": category,
        "details": details,
        "reply": reply,
    }

In [18]:
sample_message = (
    "Hi, I was just charged twice for my monthly subscription. "
    "The first charge was yesterday and the second one this morning. "
    "Can you fix this ASAP? I really need that money back."
)

result = customer_support_flow(sample_message)

print("=== Step 1: Predicted Category ===")
print(result["category"])
print("\n=== Step 2: Extracted Details (JSON) ===")
print(json.dumps(result["details"], indent=2))
print("\n=== Step 3: Final Support Reply ===")
print(result["reply"])

=== Step 1: Predicted Category ===
billing

=== Step 2: Extracted Details (JSON) ===
{
  "summary": "Customer was charged twice for their monthly subscription and requests a refund.",
  "important_details": [
    "Two charges for the monthly subscription",
    "First charge occurred yesterday",
    "Second charge occurred this morning",
    "Customer requests urgent refund"
  ],
  "urgency": "high"
}

=== Step 3: Final Support Reply ===
Hi, I’m sorry for the double charge and the trouble it’s caused. I’ve flagged this as urgent and will start the refund process right away. You should see the refund on your account within 3-5 business days. If you don’t, please let us know immediately. Thanks for your patience!


In [ ]:
print("Customer Support Chatbot")
print("Type 'quit' to end the conversation.")

turn = 0

while True:
    user_message = input("You: ")
    if user_message.strip().lower() in {"quit", "exit"}:
        print("Bot: Thank you for contacting support. Goodbye!")
        break

    # Generate only the polished support reply
    result = customer_support_flow(user_message)

    print(f"Bot: {result['reply']}\n")

    turn += 1

In [22]:
messages = [
    "Hi, I think I was charged twice for my monthly subscription.",
    "I also can't see my latest invoice in my account.",
    "Thanks. One more thing: how do I update my credit card on file?"
]

print("=== Scripted 3-Turn Customer Support Conversation ===\n")

for msg in messages:
    print("You:", msg)
    result = customer_support_flow(msg)
    print("Bot:", result['reply'])
    print()

=== Scripted 3-Turn Customer Support Conversation ===

You: Hi, I think I was charged twice for my monthly subscription.
Bot: Hi! Thanks for reaching out. I’m sorry to hear about the possible double charge. I’ll review your account right away to check the billing details. Meanwhile, could you please confirm the date and amount of the charges you see? This will help us resolve the issue quickly. We’ll get back to you as soon as we have an update.

You: I also can't see my latest invoice in my account.
Bot: I'm sorry you're having trouble seeing your latest invoice. Sometimes it can take a little time to appear in your account. I’ll check on our end to make sure it was generated correctly. Meanwhile, please try refreshing your account page or logging out and back in. If it still doesn’t show up in a day, let us know and we’ll help you get a copy right away.

You: Thanks. One more thing: how do I update my credit card on file?
Bot: Hi! To update your credit card on file, please log into y